# Script de seleção de caracteristicas dos clusters formados nas bases de dados do ENEM

**Autor**: Rafael Victor Araujo Bernardes - rafaelvictor.bernardes@gmail.com

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2

In [2]:
# Variáveis de controle

# ano = '2019'
# ano = '2020'
# ano = '2021'
# ano = '2022'
ano = '2023'

DATASET_ENEM_PATH = f'D:/BASES_PRE_PROCESSADAS/PRE_PROCESSADOS_ENEM_{ano}.csv'

In [3]:
microdadosEnem = pd.read_csv(DATASET_ENEM_PATH, sep=',', encoding='ISO-8859-1')

In [4]:
microdadosEnem.shape

(2678264, 169)

In [5]:
microdadosEnem.columns

Index(['MEDIA_NOTAS', 'TP_FAIXA_ETARIA_1', 'TP_FAIXA_ETARIA_2',
       'TP_FAIXA_ETARIA_3', 'TP_FAIXA_ETARIA_4', 'TP_FAIXA_ETARIA_5',
       'TP_FAIXA_ETARIA_6', 'TP_FAIXA_ETARIA_7', 'TP_FAIXA_ETARIA_8',
       'TP_FAIXA_ETARIA_9',
       ...
       'Q024_D', 'Q024_E', 'Q025_A', 'Q025_B', 'MACRO_REGIAO_CENTRO_OESTE',
       'MACRO_REGIAO_NORDESTE', 'MACRO_REGIAO_NORTE', 'MACRO_REGIAO_SUDESTE',
       'MACRO_REGIAO_SUL', 'K_Cluster'],
      dtype='object', length=169)

In [6]:
def avaliar_melhor_k(X, y, max_k=20):
    scores = []
    
    for k in range(1, min(max_k, X.shape[1]) + 1):
        feature_selector = SelectKBest(score_func=chi2, k=k)  # Usando chi2 para ser consistente
        X_new = feature_selector.fit_transform(X, y)
        
        # Usando RandomForest para avaliar a performance das features selecionadas
        model = RandomForestClassifier(random_state=42)
        score = cross_val_score(model, X_new, y, cv=5).mean()  # Validação cruzada com 5 folds
        scores.append((k, score))
    
    # Encontrar o valor de k que maximiza o score
    melhor_k = max(scores, key=lambda item: item[1])[0]
    
    print(f"Melhor valor de k: {melhor_k}")
    return melhor_k, scores

In [7]:
microdados_enem_features = [col for col in microdadosEnem.columns if col != 'MEDIA_NOTAS' and col != 'K_Cluster']

In [10]:
# Amostragem estratificada para acelerar a análise (1% dos dados)

microdados_sampled, _ = train_test_split(microdadosEnem, test_size=0.99, stratify=microdadosEnem['K_Cluster'], random_state=42)

In [11]:
for cluster in np.sort(microdados_sampled.K_Cluster.unique()):
    
    # Aproximar o melhor valor de k
    labels_binarias_sampled = (microdados_sampled['K_Cluster'] == cluster).astype(int)
    X_sampled = microdados_sampled[microdados_enem_features]
    melhor_k_aproximado, resultados = avaliar_melhor_k(X_sampled, labels_binarias_sampled)
    print(f"Cluster {cluster}: Melhor valor de aproximado de k é: {melhor_k_aproximado}")
    
    # Selecionar features com o melhor valor de k aproximado
    labels_binarias = (microdadosEnem['K_Cluster'] == cluster).astype(int)
    feature_selector = SelectKBest(score_func=chi2, k=melhor_k_aproximado)
    X = microdadosEnem[microdados_enem_features]
    feature_selector.fit_transform(X, labels_binarias)
    
    # Exibir as features selecionadas
    print(f"Cluster: {cluster:d}; Features: {X.columns[feature_selector.get_support()]}")

Melhor valor de k: 20
Cluster 0: Melhor valor de aproximado de k é: 20
Cluster: 0; Features: Index(['TP_FAIXA_ETARIA_3', 'TP_ST_CONCLUSAO_1', 'TP_ST_CONCLUSAO_2',
       'TP_ST_CONCLUSAO_3', 'TP_ESCOLA_1', 'TP_ESCOLA_2', 'Q006_B', 'Q010_A',
       'Q010_B', 'Q013_A', 'Q013_B', 'Q014_A', 'Q014_B', 'Q016_A', 'Q016_B',
       'Q018_B', 'Q024_A', 'Q024_B', 'Q025_A', 'MACRO_REGIAO_NORDESTE'],
      dtype='object')
Melhor valor de k: 19
Cluster 1: Melhor valor de aproximado de k é: 19
Cluster: 1; Features: Index(['TP_ST_CONCLUSAO_1', 'TP_ST_CONCLUSAO_2', 'TP_ESCOLA_1', 'TP_ESCOLA_2',
       'Q006_B', 'Q008_B', 'Q008_C', 'Q010_A', 'Q010_B', 'Q013_A', 'Q013_B',
       'Q014_A', 'Q014_B', 'Q016_A', 'Q016_B', 'Q018_B', 'Q021_B', 'Q022_B',
       'Q024_A'],
      dtype='object')
Melhor valor de k: 20
Cluster 2: Melhor valor de aproximado de k é: 20
Cluster: 2; Features: Index(['TP_ESCOLA_3', 'Q001_G', 'Q002_G', 'Q003_E', 'Q004_E', 'Q006_L',
       'Q007_B', 'Q008_D', 'Q008_E', 'Q010_C', 'Q012_C',